# Bedrock AgentCore Gateway를 사용하여 OpenAPI API를 MCP 도구로 변환

## 개요
Bedrock AgentCore Gateway provides customers a way to turn their existing APIs into fully-managed MCP servers without needing to manage infra or hosting. Customers can bring OpenAPI spec in JSON or YAML. We will demonstrate a customer service agent using enterprise support apis secured by OAuth2.

Gateway 워크플로우는 에이전트를 외부 도구에 연결하기 위해 다음 단계를 포함합니다:* **Gateway용 도구 생성** - REST API용 OpenAPI 사양과 같은 스키마를 사용하여 도구를 정의합니다. 그러면 OpenAPI 사양이 Amazon Bedrock AgentCore에 의해 파싱되어 Gateway를 생성합니다.
* **Create a Gateway endpoint** - Create the gateway that will serve as the MCP entry point with inbound authentication.
* **Add targets to your Gateway** - Configure the OpenAPI targets that define how the gateway routes requests to specific tools. All the APIs that part of OpenAPI file will become an MCP-compatible tool, and will be made available through your Gateway endpoint URL. Configure outbound authorization using Oauth for each OpenAPI Gateway target. 
* **Update your agent code** - Connect your agent to the Gateway endpoint to access all configured tools through the unified MCP interface.

![작동 방식](images/openapis-oauth-gateway.png)

### 튜토리얼 세부 정보


| 정보                 | 세부 사항                                                 |
|:---------------------|:----------------------------------------------------------|
| 튜토리얼 유형           | 대화형                                                    |
| AgentCore 구성 요소    | AgentCore Gateway, AgentCore Identity                     |
| Agentic Framework    | Strands Agent                                             |
| Gateway 대상 유형      | OpenAPI                                                   |
| Agent                | Customer support agent                                    |
| Inbound Auth IdP     | Okta                                                      |
| 아웃바운드 인증        | OAuth                                                     |
| LLM 모델             | Anthropic Claude Sonnet 3.7, Amazon Nova Pro              |
| 튜토리얼 구성 요소      | AgentCore Gateway 생성 및 AgentCore Gateway 호출            |
| 튜토리얼 분야          | 교차 분야                                                 |
| 예제 복잡성           | 쉬움                                                      |
| 사용된 SDK           | boto3                                                     |

튜토리얼의 첫 번째 부분에서는 몇 가지 AmazonCore Gateway 대상을 생성할 것입니다

### 튜토리얼 아키텍처
이 튜토리얼에서는 OpenAPI yaml/json 파일에 정의된 작업을 MCP 도구로 변환하고 Bedrock AgentCore Gateway에서 호스팅할 것입니다.시연 목적으로 지원 티켓과 관련된 쿼리에 답변하는 고객 지원 에이전트를 구축할 것입니다. 에이전트는 Zendesk 지원 API의 OpenAPI를 사용합니다. 솔루션은 Amazon Bedrock 모델을 사용하는 Langchain 에이전트를 사용합니다

## 전제 조건

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Jupyter notebook with Python 3.10+
* uv
* AWS 자격 증명
* Okta
    - client_id
    - client_secret
    - Your Okta domain (e.g., dev-123456.okta.com)
    - An OAuth2 authorization server ID (often default)

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
# Set AWS credentials if not using SageMaker notebooks
import os
# os.environ['AWS_ACCESS_KEY_ID'] = '' # 액세스 키 설정
# os.environ['AWS_SECRET_ACCESS_KEY'] = '' # 시크릿 키 설정
os.environ['AWS_DEFAULT_REGION'] = os.environ.get('AWS_REGION', 'us-east-1')

In [ ]:
import os
import sys

# 현재 스크립트의 디렉토리 가져오기if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # __file__이 정의되지 않은 경우의 대안 (예: Jupyter)

# utils.py가 포함된 디렉토리로 이동 (한 단계 위)utils_dir = os.path.abspath(os.path.join(current_dir, '../..'))

# sys.path에 추가sys.path.insert(0, utils_dir)

# 이제 utils를 가져올 수 있습니다import utils

In [ ]:
#### Gateway가 맡을 IAM 역할 생성import utils

agentcore_gateway_iam_role = utils.create_agentcore_gateway_role("sample-lambdagateway")
print("Agentcore 게이트웨이 역할 ARN: ", agentcore_gateway_iam_role['Role']['Arn'])

# 게이트웨이 인바운드 권한 부여를 위한 Okta 구성

Okta OAuth 권한 부여자를 생성하는 단계는 다음과 같습니다.

* [여기](https://developer.okta.com/docs/guides/implement-grant-type/clientcreds/main/)의 지침을 따르고 클라이언트 자격 증명 권한 부여 유형으로 애플리케이션을 생성합니다. Okta 구독이 있는 경우 관리 콘솔에 로그인하여 [여기](https://developer.okta.com/docs/guides/implement-grant-type/clientcreds/main/)에 설명된 단계를 수행할 수 있습니다. Okta 구독이 없는 경우 무료 체험판에 가입해야 합니다.
* 관리 -> 애플리케이션 -> 비밀 키를 사용하여 클라이언트 생성으로 이동합니다. 토큰 요청에서 소유 증명 증명 요구(DPoP) 헤더를 비활성화합니다.
* Okta 관리 -> 보안 -> API로 이동합니다. 기본 권한 부여 서버를 사용하고 추가 범위(예: InvokeGateway)를 수정합니다. 선택적으로 액세스 정책 및 클레임을 추가할 수 있습니다.
* 사용자 지정 범위를 정의합니다. 권한 부여 서버 이름을 클릭하고 범위 탭으로 이동한 후 "범위 추가"를 클릭합니다.
* 구성이 완료되면 아래와 같이 게이트웨이 생성을 위한 사용자 지정 JWT 권한 부여자를 구성하기 위해 기본 권한 부여 서버의 메타데이터 URI(검색 URI라고도 함)와 클라이언트 ID/비밀번호가 필요합니다.

# Create the Gateway with Okta authorizer for inbound authorization

In [ ]:
import boto3
from pprint import pprint
gateway_client = boto3.client('bedrock-agentcore-control', region_name = os.environ['AWS_DEFAULT_REGION'])

OKTA_DISCOVERY_URL="https://<YOUR OKTA DOMAIN>/oauth2/<Your app>/.well-known/openid-configuration"
OKTA_AUDIENCE="<Your audience>" # e.g. MCPGateway. It should match with your configuration in Okta

auth_config = {
        "customJWTAuthorizer": {
            "allowedAudience": [OKTA_AUDIENCE],
            "discoveryUrl": OKTA_DISCOVERY_URL
        }
}
create_response = gateway_client.create_gateway(name='OpenAPIOktaGwy2',
    roleArn = agentcore_gateway_iam_role['Role']['Arn'], # IAM 역할은 Gateway를 생성/나열/가져오기/삭제할 권한이 있어야 합니다 
    protocolType='MCP',
    authorizerType='CUSTOM_JWT',
    authorizerConfiguration=auth_config, 
    description='AgentCore Gateway created from sdk with Okta Authorizer'
)
pprint(create_response)
# GatewayTarget 생성에 사용되는 GatewayID 검색gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]

# Transforming Zendesk support APIs into MCP tools using Bedrock AgentCore Gateway

### Create outbound auth credentials provider

In [ ]:
from botocore.config import Config
ZENDESK_DOMAIN="<Zendek domain url>"
ZENDESK_AUTH_ENDPOINT="https://<Zendeskl-domain>/oauth/authorizations/new"
ZENDESK_TOKEN_ENDPOINT="https://<Zendesk-domain>/oauth/tokens"
ZENDESK_CLIENT_ID="" # Your Zendesk OAuth client -  client id 
ZENDESK_SECRET=""  # Your Zendesk OAuth client -  client id 

sdk_config = Config(
    region_name=os.environ['AWS_DEFAULT_REGION'],
    retries={"max_attempts": 2, "mode": "standard"},
)

acps = boto3.client(
    service_name="bedrock-agentcore-control",
    config=sdk_config,
)

provider_config= {
    "customOauth2ProviderConfig": {
         "oauthDiscovery": {
             "authorizationServerMetadata": {
                 "issuer": ZENDESK_DOMAIN,
                 "authorizationEndpoint": ZENDESK_AUTH_ENDPOINT,
                 "tokenEndpoint": ZENDESK_TOKEN_ENDPOINT,
                 "responseTypes": ["token"]
             }
         },
         "clientId": ZENDESK_CLIENT_ID,
         "clientSecret": ZENDESK_SECRET
     }
 }

response = acps.create_oauth2_credential_provider(
    name="ZendeskOAuthTokenCfg", 
    credentialProviderVendor="CustomOauth2", 
    oauth2ProviderConfigInput=provider_config
)

pprint(response)
credentialProviderARN = response['credentialProviderArn']
pprint(f"Egress Credentials provider ARN, {credentialProviderARN}")

### Create an OpenAPI target 

#### Upload the Zendesk support OpenAPI yaml file in S3

In [ ]:
# Create an S3 client
session = boto3.session.Session()
s3_client = session.client('s3')
sts_client = session.client('sts')

# Retrieve AWS account ID and region
account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
# Define parameters
bucket_name = '' # Your s3 bucket to upload the OpenAPI json file.
file_path = 'openapi-specs/Zendesk-support-apis.yaml'
object_key = 'Zendesk-support-apis.yaml'
# Upload the file using put_object and read response
try:
    with open(file_path, 'rb') as file_data:
        response = s3_client.put_object(Bucket=bucket_name, Key=object_key, Body=file_data)

    # Construct the ARN of the uploaded object with account ID and region
    openapi_s3_uri = f's3://{bucket_name}/{object_key}'
    print(f'Uploaded object S3 URI: {openapi_s3_uri}')
except Exception as e:
    print(f'Error uploading file: {e}')

#### Create the gateway target

OpenAPI 파일의 서버 URL이 자체 엔드포인트 URL을 가리키는지 확인하세요. Gateway는 OpenAPI 파일에서 서버 URL을 읽고 엔드포인트를 호출합니다. S3에 업로드하기 전에 이 변경 작업을 수행해야 합니다.

In [ ]:
# S3 Uri for OpenAPI spec file
openapi_s3_target_config = {
    "mcp": {
          "openApiSchema": {
              "s3": {
                  "uri": openapi_s3_uri
              }
          }
      }
}

credential_config = [
    {
        "credentialProviderType" : "OAUTH",
        "credentialProvider": {
            "oauthCredentialProvider": {
                "providerArn": credentialProviderARN, 
                "scopes": ["tickets:read", "read", "tickets:write", "write"] 
            }
        }
    }
  ]

target_name="DemoOpenAPIGW"
response = gateway_client.create_gateway_target(
    gatewayIdentifier=gatewayID,
    name=target_name,
    description='OpenAPI Target with S3Uri using SDK',
    targetConfiguration=openapi_s3_target_config,
    credentialProviderConfigurations=credential_config)

# Printing the request ID and timestamp for you to report the defects. Please include them while reporting issues/defects  
response_metadata = response['ResponseMetadata']

# Strands Agent에서 Bedrock AgentCore Gateway 호출

Strands 에이전트는 Model Context Protocol(MCP) 사양을 구현하는 Bedrock AgentCore Gateway를 통해 AWS 도구와 원활하게 통합됩니다. 이 통합은 AI 에이전트와 AWS 서비스 간의 안전하고 표준화된 통신을 가능하게 합니다.

핵심적으로 Bedrock AgentCore Gateway는 기본적인 MCP API인 ListTools와 InvokeTools를 노출하는 프로토콜 준수 Gateway 역할을 합니다. 이러한 API를 통해 MCP 호환 클라이언트나 SDK는 안전하고 표준화된 방식으로 사용 가능한 도구를 발견하고 상호 작용할 수 있습니다. Strands 에이전트가 AWS 서비스에 액세스해야 할 때 이러한 MCP 표준화된 엔드포인트를 사용하여 Gateway와 통신합니다.

Gateway의 구현은 (MCP 권한 부여 사양)[https://modelcontextprotocol.org/specification/draft/basic/authorization]을 엄격히 준수하여 강력한 보안과 액세스 제어를 보장합니다. 이는 Strands 에이전트의 모든 도구 호출이 권한 부여 단계를 거치므로 강력한 기능을 활성화하면서 보안을 유지한다는 것을 의미합니다.

예를 들어, Strands 에이전트가 MCP 도구에 액세스해야 할 때 먼저 ListTools를 호출하여 사용 가능한 도구를 발견한 다음 InvokeTools를 사용하여 특정 작업을 실행합니다. Gateway는 필요한 모든 보안 검증, 프로토콜 변환 및 서비스 상호 작용을 처리하여 전체 프로세스를 원활하고 안전하게 만듭니다.

이러한 아키텍처 접근 방식은 MCP 사양을 구현하는 모든 클라이언트나 SDK가 Gateway를 통해 AWS 서비스와 상호 작용할 수 있음을 의미하며, AI 에이전트 통합을 위한 다재다능하고 미래 지향적인 솔루션이 됩니다.

# Request the access token from Okta for inbound authorization

In [ ]:
print("Requesting the access token from Okta authorizer")
import requests
from requests.auth import HTTPBasicAuth

# Replace with your actual values
OKTA_DOMAIN = "Your Okta domain URL"
AUTH_SERVER_ID = "Okta app id"
CLIENT_ID = "<Okta client credentials client id>"
CLIENT_SECRET = "<Okta client credentials secret>"

TOKEN_URL = f"{OKTA_DOMAIN}/oauth2/{AUTH_SERVER_ID}/v1/token"

response = requests.post(
    TOKEN_URL,
    auth=HTTPBasicAuth(CLIENT_ID, CLIENT_SECRET),
    headers={"Content-Type": "application/x-www-form-urlencoded"},
    data={"grant_type": "client_credentials", "scope": "InvokeGateway"}
)

if response.status_code == 200:
    token = response.json()["access_token"]
    print("Access Token:", token)
else:
    print("Failed to get token:", response.status_code, response.text)

# Ask customer support agent with Zendesk support APIs using Bedrock AgentCore Gateway

In [ ]:
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client 
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent

def create_streamable_http_transport():
    return streamablehttp_client(gatewayURL,headers={"Authorization": f"Bearer {token}"})

client = MCPClient(create_streamable_http_transport)

## ~/.aws/credentials에 구성된 IAM 그룹/사용자는 Bedrock 모델에 액세스할 수 있어야 합니다yourmodel = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",
    temperature=0.7,
)

In [ ]:
from strands import Agent
import logging


# 루트 strands 로거를 구성합니다. 문제를 디버깅하는 경우 DEBUG로 변경하세요.logging.getLogger("strands").setLevel(logging.INFO)

# 로그를 보기 위한 핸들러 추가logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

with client:
# listTools 호출    tools = client.list_tools_sync()
# 모델과 도구로 에이전트 생성    agent = Agent(model=yourmodel,tools=tools) ## 원하는 모델로 교체할 수 있습니다
    #print(f"에이전트에 로드된 도구는 {agent.tool_names}")
    #print(f"Tools configuration in the agent are {agent.tool_config}")
    
    # 샘플 프롬프트로 에이전트를 호출합니다. 이는 MCP listTools만 호출하고 LLM이 액세스할 수 있는 도구 목록을 검색합니다. 아래는 실제로 어떤 도구도 호출하지 않습니다.
    #agent("안녕하세요, 사용 가능한 모든 도구를 나열해 주실 수 있나요?")
    agent("Count the number of support tickets")
# MCP 도구를 명시적으로 호출합니다. MCP 도구 이름과 인수는 AWS Lambda 함수 또는 OpenAPI/Smithy API와 일치해야 합니다    result = client.call_tool_sync(
    tool_use_id="count-tickets-1", # 이것을 고유 식별자로 교체할 수 있습니다. 
    name="DemoOpenAPIGW___CountTickets", # 이는 AWS Lambda 대상 유형을 기반으로 한 도구 이름입니다. 대상 이름에 따라 변경됩니다
    )
    #Print the MCP Tool response
    print(f"도구 호출 결과: {result['content'][0]['text']}")


# 정리
IAM 역할, IAM 정책, 자격 증명 공급자, AWS Lambda 함수, Cognito 사용자 풀, S3 버킷과 같은 추가 리소스도 생성되며, 정리의 일부로 수동으로 삭제해야 할 수 있습니다. 이는 실행하는 예제에 따라 다릅니다.

## 게이트웨이 삭제 (선택사항)

In [ ]:
import utils
utils.delete_gateway(gateway_client,gatewayID)